In [1]:
import torch
from torch import nn
from torch.autograd import Variable

In [23]:
import os
import os.path

import shutil

directory = "geoguessr/"
directory_to = "geoguessr_filtered/"

os.makedirs(directory_to, exist_ok=True)

filenames = []

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if (os.path.isdir(f)):
        i = 0
        for image in os.listdir(f):
            if i % 200 == 0:    
                filenames.append(f"{f}/{image}")
                os.makedirs(f"{directory_to}/{f.split('/')[1]}", exist_ok=True)
                shutil.copyfile(f"{f}/{image}", f"{directory_to}/{f.split('/')[1]}/{image}")
            i+=1

In [20]:
import geoLSTM

2022-06-02 22:14:37.888988: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-02 22:14:37.889023: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [142]:
from shapely.geometry import Point, Polygon
from matplotlib import pyplot as plt
from math import sin, cos, sqrt, atan2, radians

# !pip install gmaps
import shapely
import pickle
import random
import numpy as np
import gmaps, os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from ipywidgets.embed import embed_minimal_html
import webbrowser
import argparse

from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, LSTM
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D, GlobalMaxPool3D
from tensorflow.keras.applications.resnet50 import ResNet50

In [143]:
filenames
fn1 = list(map(lambda x: "/".join(x.split("/")[1:]), filenames))
print(fn1)

['Japan/canvas_1629498589.jpg', 'Japan/canvas_1629687061.jpg', 'Japan/canvas_1630103831.jpg', 'Japan/canvas_1629324654.jpg', 'Japan/canvas_1629991557.jpg', 'Japan/canvas_1629428217.jpg', 'Japan/canvas_1629260090.jpg', 'Japan/canvas_1630020009.jpg', 'Japan/canvas_1630142164.jpg', 'Japan/canvas_1629992051.jpg', 'Japan/canvas_1630143268.jpg', 'Japan/canvas_1629750162.jpg', 'Japan/canvas_1629509519.jpg', 'Japan/canvas_1629973056.jpg', 'Japan/canvas_1629994129.jpg', 'Japan/canvas_1629538319.jpg', 'Japan/canvas_1629709643.jpg', 'Japan/canvas_1629399503.jpg', 'Japan/canvas_1629698128.jpg', 'Japan/canvas_1629535284.jpg', 'France/canvas_1630041998.jpg', 'France/canvas_1630283666.jpg', 'France/canvas_1630285286.jpg', 'France/canvas_1629785749.jpg', 'France/canvas_1630043816.jpg', 'France/canvas_1630009087.jpg', 'France/canvas_1630030677.jpg', 'France/canvas_1629740965.jpg', 'France/canvas_1629836167.jpg', 'France/canvas_1630282593.jpg', 'France/canvas_1630216681.jpg', 'France/canvas_1629595064.j

In [144]:
classes = list(set(map(lambda x: x.split("/")[1], filenames)))
numclasses = len(classes)
print(numclasses)
classes = dict(zip(classes, range(numclasses)))
classes

124


{'Monaco': 0,
 'Greenland': 1,
 'Romania': 2,
 'Thailand': 3,
 'Germany': 4,
 'Mozambique': 5,
 'Uganda': 6,
 'Turkey': 7,
 'Svalbard and Jan Mayen': 8,
 'Myanmar': 9,
 'New Zealand': 10,
 'Argentina': 11,
 'Belarus': 12,
 'Estonia': 13,
 'Tunisia': 14,
 'Guatemala': 15,
 'Japan': 16,
 'Puerto Rico': 17,
 'Tanzania': 18,
 'Russia': 19,
 'Martinique': 20,
 'Costa Rica': 21,
 'Bhutan': 22,
 'United Kingdom': 23,
 'Paraguay': 24,
 'Palestine': 25,
 'Singapore': 26,
 'Denmark': 27,
 'Vietnam': 28,
 'South Georgia and South Sandwich Islands': 29,
 'Gibraltar': 30,
 'Egypt': 31,
 'Ecuador': 32,
 'Serbia': 33,
 'Italy': 34,
 'Peru': 35,
 'India': 36,
 'Andorra': 37,
 'Belgium': 38,
 'Aland': 39,
 'Indonesia': 40,
 'South Sudan': 41,
 'Luxembourg': 42,
 'Croatia': 43,
 'Brazil': 44,
 'Colombia': 45,
 'Albania': 46,
 'Cambodia': 47,
 'Kyrgyzstan': 48,
 'Philippines': 49,
 'Nepal': 50,
 'Ghana': 51,
 'Chile': 52,
 'Iceland': 53,
 'Macao': 54,
 'Jersey': 55,
 'United Arab Emirates': 56,
 'Montene

In [145]:


def readData(fileNames, dataDir):
    '''
    Takes a list of location folder names and ouputs a list of input image vectors and ouput categorical grid vector pairs.
    fileNames should look like: 60+48.4271513,-110.5611851
    '''
#     numClasses = self.model.layers[-1].output_shape[-1]
#     inputShape = self.model.layers[0].input_shape[2:4]
    # load three images in a folder as arrays
    X = np.array(list(map(lambda x: np.array(load_img(dataDir+x)), fileNames)))
    # load grid numbers from folder names and convert them to categorical ouput vectors
    y = tf.keras.utils.to_categorical(list(map(lambda x: classes[x.split("/")[0]], fileNames)),
                              num_classes=numclasses)
    return X,y

def dataGen(fileNames, dataDir, batchSize=10, infinite=True):
    '''
    Takes a list of location folder names and ouputs
    a list of input image vectors and ouput categorical grid vector pairs in batches.
    The function is essentially used as a generator that calls readData in datches.
    Inifinit: Tells the function to stop or keep going once the list of file names has been iterated through
    fileNames should look like: 60+48.4271513,-110.5611851
    '''
    totalBatches = len(fileNames)/batchSize
    counter=0
    while(True):
        prev = batchSize*counter
        nxt = batchSize*(counter+1)
        counter+=1
        yield readData(fileNames[prev:nxt],dataDir)
        if counter>=totalBatches:
            if infinite:
                counter=0
            else:
                break

In [146]:
it = dataGen(fn1, directory_to)

In [147]:
X, y = next(it)

In [148]:
outputShape = y.shape
outputShape

(10, 124)

In [149]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [150]:
inputShape = X.shape
inputShape = (1, *inputShape[1:])
inputShape

(1, 662, 1536, 3)

In [151]:
X

array([[[[188, 184, 185],
         [188, 184, 185],
         [188, 184, 185],
         ...,
         [180, 178, 179],
         [180, 178, 179],
         [180, 178, 179]],

        [[188, 184, 185],
         [188, 184, 185],
         [188, 184, 185],
         ...,
         [180, 178, 179],
         [180, 178, 179],
         [180, 178, 179]],

        [[188, 184, 185],
         [188, 184, 185],
         [188, 184, 185],
         ...,
         [180, 178, 179],
         [180, 178, 179],
         [180, 178, 179]],

        ...,

        [[163, 161, 164],
         [164, 162, 165],
         [165, 163, 166],
         ...,
         [ 14,  17,   6],
         [ 11,  14,   3],
         [  9,  12,   1]],

        [[166, 164, 167],
         [166, 164, 167],
         [167, 165, 168],
         ...,
         [ 18,  21,  10],
         [ 16,  19,   8],
         [ 14,  17,   6]],

        [[168, 166, 169],
         [169, 167, 170],
         [170, 168, 171],
         ...,
         [ 26,  29,  18],
        

In [152]:
model=None

In [155]:



convnet = tf.keras.Sequential()

# restnet = ResNet50(include_top=False, weights='imagenet', input_shape=inputShape[1:])
# # Freeze model
# restnet.trainable = False
# convnet.add(restnet)

# Use trainable CNN
convnet.add(Conv2D(128, (3,3), input_shape=inputShape[1:],
    padding='same', activation='relu'))
convnet.add(Conv2D(128, (3,3), padding='same', activation='relu'))
convnet.add(BatchNormalization(momentum=.6))
convnet.add(MaxPool2D())
convnet.add(Conv2D(64, (3,3), padding='same', activation='relu'))
convnet.add(Conv2D(64, (3,3), padding='same', activation='relu'))
convnet.add(BatchNormalization(momentum=.6))
convnet.add(MaxPool2D())
convnet.add(Conv2D(64, (3,3), padding='same', activation='relu'))
convnet.add(Conv2D(64, (3,3), padding='same', activation='relu'))
convnet.add(BatchNormalization(momentum=.6))
convnet.add(MaxPool2D())
convnet.add(Conv2D(512, (3,3), padding='same', activation='relu'))
convnet.add(Conv2D(512, (3,3), padding='same', activation='relu'))
convnet.add(BatchNormalization(momentum=.6))


convnet.add(GlobalMaxPool2D())
model = tf.keras.Sequential()
# Connect the CNN to an LSTM
model.add(convnet)
# model.add(LSTM(64))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(outputShape[1], activation='softmax'))
model.compile(loss=tf.keras.losses.categorical_crossentropy,
          optimizer=tf.keras.optimizers.Adam(), metrics=['categorical_accuracy'])
model.summary()


Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_45 (Sequential)  (None, 512)               2994048   
                                                                 
 dense_54 (Dense)            (None, 1024)              525312    
                                                                 
 dropout_33 (Dropout)        (None, 1024)              0         
                                                                 
 dense_55 (Dense)            (None, 512)               524800    
                                                                 
 dropout_34 (Dropout)        (None, 512)               0         
                                                                 
 dense_56 (Dense)            (None, 128)               65664     
                                                                 
 dropout_35 (Dropout)        (None, 128)             

In [156]:
total_loss = 0

def save(saveFolder, modelNumber=0):
    '''
    Saves model to specified folder with specified number along with loss
    saveFolder: Folder to save trained model to eg: "models"
    '''
    if total_loss==-1:
        print("Cannot save untrained model!!!")
    else:
        print("\nSaving model {} with loss {} at {}".format(modelNumber,
                                                            total_loss,
                                                           saveFolder))
        model.save(saveFolder + '/model_{}_{}.h5'.format(total_loss,
                                                         modelNumber))


class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
    '''
    Custom callback used to save model at the end of a training batch.
    '''
    def __init__(self, saveFolder, modelNumber):
        '''
        gGuess: Instance of Geoguessr
        saveFolder: Name of floder to save to
        modelNumber: A number to add to saved model file name.
        Used to find how far into training a model was saved
        '''
        super(LossAndErrorPrintingCallback, self).__init__()
        self.modelNumber =  modelNumber
        self.saveFolder = saveFolder

    def on_epoch_end(self, epoch, logs=None):
        '''
        Update model loss every few epochs
        '''
        total_loss = round(float(logs['loss']),3)

    def on_train_end(self, logs={}):
        '''
        Save model at the end of training
        '''
        print("Training sucessfull!!")
        save(self.saveFolder, self.modelNumber)

In [ ]:

epochs=20

print("Getting data from directory: {}".format(directory_to))
accuracy = []
loss = []
saveFolder = "models"
cnt = 0
for X,y in dataGen(fn1, directory_to, batchSize=11, infinite=False):
    callBack = [LossAndErrorPrintingCallback(saveFolder, cnt)]
    print("Read {} points. Training now".format(len(X)))
    evalutaion = model.fit(X,y,
                                epochs=epochs, steps_per_epoch = len(X),
                                callbacks=callBack)
    accuracy += evalutaion.history['categorical_accuracy']
    loss += evalutaion.history['loss']
    cnt += 1
if plot:
    plt.plot(accuracy)
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.show()

    plt.plot(loss)
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()


Getting data from directory: geoguessr_filtered/
Read 11 points. Training now
Epoch 1/20
